# Obspy deals with station inventory XML files

## Fei Zhang

2020-02-11

## How to build a stationxml file (with extra elements) from scratch

In [1]:
# https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html

import obspy
from obspy import Inventory, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL

from obspy.core.util import AttribDict


# We'll first create all the various objects. These strongly follow the
# hierarchy of StationXML files.
inv = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="ObsPy-Tutorial")

net = Network(
    # This is the network code according to the SEED standard.
    code="XX",
    # A list of stations. We'll add one later.
    stations=[],
    description="A test stations.",
    # Start-and end dates are optional.
    start_date=obspy.UTCDateTime(2016, 1, 2))

sta = Station(
    # This is the station code according to the SEED standard.
    code="ABC",
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    creation_date=obspy.UTCDateTime(2016, 1, 2),
    site=Site(name="First station"),
    )

cha = Channel(
    # This is the channel code according to the SEED standard.
    code="HHZ",
    # This is the location code according to the SEED standard.
    location_code="",
    # Note that these coordinates can differ from the station coordinates.
    latitude=1.0,
    longitude=2.0,
    elevation=345.0,
    depth=10.0,
    azimuth=0.0,
    dip=-90.0,
    sample_rate=200)

# By default this accesses the NRL online. Offline copies of the NRL can
# also be used instead
nrl = NRL()
# The contents of the NRL can be explored interactively in a Python prompt,
# see API documentation of NRL submodule:
# http://docs.obspy.org/packages/obspy.clients.nrl.html
# Here we assume that the end point of data logger and sensor are already
# known:
response = nrl.get_response( # doctest: +SKIP
    sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
    datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])


# Now tie it all together.
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

# And finally write it to a StationXML file. We also force a validation against
# the StationXML schema to ensure it produces a valid StationXML file.
#
# Note that it is also possible to serialize to any of the other inventory
# output formats ObsPy supports.
inv.write("station.xml", format="stationxml", validate=True)


# Fei's element
#extra="GPScorrection2.5"

extra = AttribDict({
           'GPSClockCorrection': {
                'value': True,
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}Description': 'Fei added new Block QA/QC time correction data',
                  '{http://some-page.de/xmlns/1.0}date': "2012-11-27",
                  '{http://some-page.de/xmlns/1.0}value_sec':"1.0398489"
                }
            },
          'GPSClockCorrection': {
                'value': 'Fei added new Block QA/QC time correction data',
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}date': "2012-11-28",
                  '{http://some-page.de/xmlns/1.0}value_sec':"0.9498489"
                }
            },
    
           'my_tag_2': {
                'value': u'True',
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_3': {
                'value': 1,
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_4': {
                'value': UTCDateTime('2013-01-02T13:12:14.600000Z'),
                'namespace': 'http://test.org/xmlns/0.1'
            },
           'my_attribute': {
                'value': 'my_attribute_value',
                'type': 'attribute',
                'namespace': 'http://test.org/xmlns/0.1'
            }
        })

inv.networks[0].stations[0].extra = extra
inv.write('my_inventory_extra.xml', format='STATIONXML',
          nsmap={'my_ns': 'http://test.org/xmlns/0.1',
                 'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [2]:
from obspy import read_inventory

inv = read_inventory('my_inventory_extra.xml')
print(inv.networks[0].stations[0].extra)

AttribDict({'GPSClockCorrection': AttribDict({'value': 'Fei added new Block QA/QC time correction data', 'attrib': {'{http://some-page.de/xmlns/1.0}value_sec': '0.9498489', '{http://some-page.de/xmlns/1.0}date': '2012-11-28'}, 'namespace': 'http://some-page.de/xmlns/1.0'}), 'my_attribute': AttribDict({'value': 'my_attribute_value', 'namespace': 'http://test.org/xmlns/0.1', 'type': 'attribute'}), 'my_tag_2': AttribDict({'value': 'True', 'namespace': 'http://some-page.de/xmlns/1.0'}), 'my_tag_3': AttribDict({'value': '1', 'namespace': 'http://some-page.de/xmlns/1.0'}), 'my_tag_4': AttribDict({'value': '2013-01-02T13:12:14.600000Z', 'namespace': 'http://test.org/xmlns/0.1'})})


# custom defined tags in StationXML with the Obspy Inventory¶

https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html
    

In [3]:
from obspy import Inventory, UTCDateTime
from obspy.core.inventory import Network
from obspy.core.util import AttribDict

extra = AttribDict({
           'my_tag': {
                'value': True,
                'namespace': 'http://some-page.de/xmlns/1.0',
                'attrib': {
                  '{http://some-page.de/xmlns/1.0}my_attrib1': '123.4',
                  '{http://some-page.de/xmlns/1.0}my_attrib2': '567'
                }
            },
           'my_tag_2': {
                'value': u'True',
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_3': {
                'value': 1,
                'namespace': 'http://some-page.de/xmlns/1.0'
            },
           'my_tag_4': {
                'value': UTCDateTime('2013-01-02T13:12:14.600000Z'),
                'namespace': 'http://test.org/xmlns/0.1'
            },
           'my_attribute': {
                'value': 'my_attribute_value',
                'type': 'attribute',
                'namespace': 'http://test.org/xmlns/0.1'
            }
        })

inv = Inventory([Network('XX')], 'XX')
inv[0].extra = extra
inv.write('my_inventory.xml', format='STATIONXML',
          nsmap={'my_ns': 'http://test.org/xmlns/0.1',
                 'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [4]:
from obspy import Inventory
from obspy.core.inventory import Network
from obspy.core.util import AttribDict

ns = 'http://some-page.de/xmlns/1.0'
# fxz547@vdi-n25 /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections
# $ head 7D.DE43_clock_correction.csv
csv_data = """ 
net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332
"""

my_tag = AttribDict()
my_tag.namespace = ns
my_tag.value = AttribDict()

my_tag.value.my_gpsclockcorrection = AttribDict()
my_tag.value.my_gpsclockcorrection.namespace = ns
my_tag.value.my_gpsclockcorrection.value = csv_data

my_tag.value.my_nested_tag2 = AttribDict()
my_tag.value.my_nested_tag2.namespace = ns
my_tag.value.my_nested_tag2.value = True

inv = Inventory([Network('XX')], 'XX')
inv[0].extra = AttribDict()
inv[0].extra.my_tag = my_tag
inv.write('my_inventory_nested.xml', format='STATIONXML',
          nsmap={'somepage_ns': 'http://some-page.de/xmlns/1.0'})

In [5]:
from obspy import read_inventory

inv = read_inventory('my_inventory_nested.xml')
print(inv[0].extra.my_tag.value.my_gpsclockcorrection.value)
print(inv[0].extra.my_tag.value.my_nested_tag2.value)

 
net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332

True


In [6]:
print(inv)

Inventory created at 2020-02-20T04:58:26.089201Z
	Created by: ObsPy 1.1.1
		    https://www.obspy.org
	Sending institution: XX
	Contains:
		Networks (1):
			XX
		Stations (0):

		Channels (0):



# Modify our station xml

Given original input station xml file, we want to read in and add extra metadada to it, 
then write out a new station xml file with the extra metadata stored.

In [7]:
from obspy import read_inventory
from obspy import Inventory
from obspy.core.inventory import Network
from obspy.core.util import AttribDict

ns = "https://github.com/GeoscienceAustralia/hiperseis/xmlns/1.0"


path2_myxml = "../tests/testdata/7D_2012_2013.xml"

my_inv = read_inventory(path2_myxml)

# https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.select.html#obspy.core.inventory.inventory.Inventory.select
    
selected_inv = my_inv.select(station="DE43")

print(selected_inv)

# fxz547@vdi-n25 /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections
# $ head 7D.DE43_clock_correction.csv
csv_data = """net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332
7D,DE43,2012-12-06,0.14886267972167544
7D,DE43,2012-12-07,0.04986421065501878
7D,DE43,2012-12-08,-0.049134258411637766
7D,DE43,2012-12-09,-0.14813272747829442
7D,DE43,2012-12-10,-0.24713119654495086
7D,DE43,2012-12-11,-0.3461296656116075
7D,DE43,2012-12-12,-0.4451281346782642
7D,DE43,2012-12-13,-0.5441266037449206
7D,DE43,2012-12-14,-0.6431250728115772
7D,DE43,2012-12-15,-0.7421235418782337
7D,DE43,2012-12-16,-0.8411220109448904
7D,DE43,2012-12-17,-0.9401204800115468
7D,DE43,2012-12-18,-1.0391189490782036
7D,DE43,2012-12-19,-1.13811741814486
7D,DE43,2012-12-20,-1.2371158872115164
7D,DE43,2012-12-21,-1.3361143562781734
7D,DE43,2012-12-22,-1.4351128253448298
7D,DE43,2012-12-23,-1.5341112944114863
7D,DE43,2012-12-24,-1.6331097634781426
7D,DE43,2012-12-25,-1.7321082325447996
7D,DE43,2012-12-26,-1.8311067016114562
7D,DE43,2012-12-27,-1.9301051706781127
7D,DE43,2012-12-28,-2.029103639744769
7D,DE43,2012-12-29,-2.128102108811426
7D,DE43,2012-12-30,-2.2271005778780824
7D,DE43,2013-02-26,1.3976528659016858
7D,DE43,2013-02-27,1.2852583172325265
7D,DE43,2013-02-28,1.1728637685633672
7D,DE43,2013-03-01,1.0604692198942076
7D,DE43,2013-03-02,0.9480746712250482
7D,DE43,2013-03-03,0.8356801225558886
7D,DE43,2013-03-04,0.7232855738867292
7D,DE43,2013-03-05,0.6108910252175697
7D,DE43,2013-03-06,0.49849647654841034
7D,DE43,2013-03-07,0.3861019278792508
7D,DE43,2013-03-08,0.2737073792100917
7D,DE43,2013-03-09,0.16131283054093215
7D,DE43,2013-03-10,0.048918281871772606
7D,DE43,2013-03-11,-0.06347626679738694
7D,DE43,2013-03-12,-0.17587081546654626
7D,DE43,2013-03-13,-0.288265364135706
7D,DE43,2013-03-14,-0.4006599128048651
7D,DE43,2013-03-15,-0.5130544614740249
7D,DE43,2013-03-16,-0.625449010143184
7D,DE43,2013-03-17,-0.7378435588123435
7D,DE43,2013-03-18,-0.8502381074815029
7D,DE43,2013-03-19,-0.9626326561506626
7D,DE43,2013-03-20,-1.0750272048198217
7D,DE43,2013-03-21,-1.1874217534889815
7D,DE43,2013-03-22,-1.299816302158141
7D,DE43,2013-03-23,-1.4122108508273004
7D,DE43,2013-03-24,-1.5246053994964597
7D,DE43,2013-03-25,-1.636999948165619
7D,DE43,2013-03-26,-1.7493944968347785
7D,DE43,2013-03-27,-1.8617890455039379
7D,DE43,2013-03-28,-1.9741835941730979
7D,DE43,2013-03-29,-2.0865781428422574
7D,DE43,2013-03-30,-2.1989726915114165
7D,DE43,2013-03-31,-2.311367240180576
7D,DE43,2013-04-01,-2.4237617888497356
7D,DE43,2013-04-02,-2.5361563375188947
7D,DE43,2013-04-03,-2.648550886188054
7D,DE43,2013-04-04,-2.7609454348572138
7D,DE43,2013-04-05,-2.873339983526373
7D,DE43,2013-04-06,-2.985734532195533
7D,DE43,2013-04-07,-3.0981290808646915
7D,DE43,2013-04-08,-3.210523629533852
7D,DE43,2013-04-09,-3.322918178203011
7D,DE43,2013-04-10,-3.4353127268721706
7D,DE43,2013-04-11,-3.5477072755413293
7D,DE43,2013-04-12,-3.6601018242104892
7D,DE43,2013-04-13,-3.772496372879649
7D,DE43,2013-04-14,-3.8848909215488083
7D,DE43,2013-04-15,-3.9972854702179674
7D,DE43,2013-04-16,-4.109680018887127
7D,DE43,2013-04-17,-4.2220745675562865
7D,DE43,2013-04-18,-4.334469116225446
7D,DE43,2013-04-19,-4.446863664894606
7D,DE43,2013-04-20,-4.559258213563765
7D,DE43,2013-04-21,-4.671652762232924
7D,DE43,2013-04-22,-4.784047310902084
7D,DE43,2013-04-23,-4.896441859571243
7D,DE43,2013-04-24,-5.008836408240402
7D,DE43,2013-04-25,-5.121230956909562
7D,DE43,2013-04-26,-5.233625505578722
7D,DE43,2013-04-27,-5.346020054247882
7D,DE43,2013-04-28,-5.458414602917041
7D,DE43,2013-04-29,-5.5708091515862
7D,DE43,2013-04-30,-5.683203700255358
7D,DE43,2013-05-01,-5.795598248924518
7D,DE43,2013-05-02,-5.9079927975936775
7D,DE43,2013-05-03,-6.020387346262838
7D,DE43,2013-05-04,-6.132781894931997
7D,DE43,2013-05-05,-6.2451764436011565
7D,DE43,2013-05-06,-6.3575709922703165
7D,DE43,2013-05-07,-6.469965540939476
7D,DE43,2013-05-08,-6.582360089608634
7D,DE43,2013-05-09,-6.694754638277794
7D,DE43,2013-05-10,-6.807149186946954
7D,DE43,2013-05-11,-6.919543735616113
7D,DE43,2013-05-12,-7.031938284285273
7D,DE43,2013-05-13,-7.144332832954432
7D,DE43,2013-05-14,-7.256727381623591
7D,DE43,2013-05-15,-7.369121930292752
7D,DE43,2013-05-16,-7.481516478961909
7D,DE43,2013-05-17,-7.593911027631068
7D,DE43,2013-05-18,-7.706305576300228
7D,DE43,2013-05-19,-7.818700124969389
7D,DE43,2013-05-20,-7.931094673638549
7D,DE43,2013-05-21,-8.043489222307707
7D,DE43,2013-05-22,-8.155883770976867
7D,DE43,2013-05-23,-8.268278319646027
7D,DE43,2013-05-24,-8.380672868315186
7D,DE43,2013-05-25,-8.493067416984344
7D,DE43,2013-05-26,-8.605461965653506
7D,DE43,2013-05-27,-8.717856514322664
7D,DE43,2013-05-28,-8.830251062991824
7D,DE43,2013-05-29,-8.942645611660984
7D,DE43,2013-05-30,-9.055040160330144
7D,DE43,2013-05-31,-9.167434708999302
7D,DE43,2013-06-01,-9.279829257668462
7D,DE43,2013-06-02,-9.392223806337622
7D,DE43,2013-06-03,-9.50461835500678
7D,DE43,2013-06-04,-9.61701290367594
7D,DE43,2013-06-05,-9.7294074523451
7D,DE43,2013-06-06,-9.841802001014258
7D,DE43,2013-06-07,-9.954196549683418
7D,DE43,2013-06-08,-10.066591098352578
7D,DE43,2013-06-09,-10.178985647021737
7D,DE43,2013-06-10,-10.291380195690897
7D,DE43,2013-06-11,-10.403774744360057
7D,DE43,2013-06-12,-10.516169293029215
7D,DE43,2013-06-13,-10.628563841698375
7D,DE43,2013-06-14,-10.740958390367535
7D,DE43,2013-06-15,-10.853352939036693
7D,DE43,2013-06-16,-10.965747487705853
7D,DE43,2013-06-17,-11.078142036375013
7D,DE43,2013-06-18,-11.190536585044171
7D,DE43,2013-06-19,-11.302931133713331
7D,DE43,2013-07-06,1.0530986289019317
7D,DE43,2013-07-07,1.0514936764358083
7D,DE43,2013-07-08,1.049888723969685
7D,DE43,2013-07-09,1.0482837715035613
7D,DE43,2013-07-10,1.046678819037438
7D,DE43,2013-07-11,1.0450738665713146
7D,DE43,2013-07-12,1.0434689141051912
7D,DE43,2013-07-13,1.0418639616390677
7D,DE43,2013-07-14,1.0402590091729442
7D,DE43,2013-07-15,1.0386540567068208
7D,DE43,2013-07-16,1.0370491042406973
7D,DE43,2013-07-17,1.035444151774574
7D,DE43,2013-07-18,1.0338391993084504
7D,DE43,2013-07-19,1.0322342468423271
7D,DE43,2013-07-20,1.0306292943762037
7D,DE43,2013-07-21,1.0290243419100802
7D,DE43,2013-07-22,1.0274193894439567
7D,DE43,2013-07-23,1.0258144369778333
7D,DE43,2013-07-24,1.0242094845117098
7D,DE43,2013-07-25,1.0226045320455865
7D,DE43,2013-07-26,1.020999579579463
7D,DE43,2013-07-27,1.0193946271133396
7D,DE43,2013-07-28,1.0177896746472161
7D,DE43,2013-07-29,1.0161847221810927
7D,DE43,2013-07-30,1.0145797697149692
7D,DE43,2013-07-31,1.0129748172488457
7D,DE43,2013-08-01,1.0113698647827223
7D,DE43,2013-08-02,1.0097649123165988
7D,DE43,2013-08-03,1.0081599598504756
7D,DE43,2013-08-04,1.0065550073843519
7D,DE43,2013-08-05,1.0049500549182286
7D,DE43,2013-08-06,1.0033451024521052
7D,DE43,2013-08-07,1.0017401499859817
7D,DE43,2013-08-08,1.0001351975198582
7D,DE43,2013-08-09,0.9985302450537348
7D,DE43,2013-08-10,0.9969252925876113
7D,DE43,2013-08-11,0.9953203401214878
7D,DE43,2013-08-12,0.9937153876553646
7D,DE43,2013-08-13,0.992110435189241
7D,DE43,2013-08-14,0.9905054827231177
7D,DE43,2013-08-15,0.9889005302569942
7D,DE43,2013-08-16,0.9872955777908707
7D,DE43,2013-08-17,0.9856906253247473
7D,DE43,2013-08-18,0.9840856728586239
7D,DE43,2013-08-19,0.9824807203925003
7D,DE43,2013-08-20,0.980875767926377
7D,DE43,2013-08-21,0.9792708154602536
7D,DE43,2013-08-22,0.97766586299413
7D,DE43,2013-08-23,0.9760609105280067
7D,DE43,2013-08-24,0.9744559580618832
7D,DE43,2013-08-25,0.9728510055957599
7D,DE43,2013-08-26,0.9712460531296363
7D,DE43,2013-08-27,0.9696411006635128
7D,DE43,2013-08-28,0.9680361481973895
7D,DE43,2013-08-29,0.966431195731266
7D,DE43,2013-08-30,0.9648262432651425
7D,DE43,2013-08-31,0.9632212907990192
7D,DE43,2013-09-01,0.9616163383328957
7D,DE43,2013-09-02,0.9600113858667723
7D,DE43,2013-09-03,0.9584064334006488
7D,DE43,2013-09-04,0.9568014809345253
7D,DE43,2013-09-05,0.9551965284684019
7D,DE43,2013-09-06,0.9535915760022786
7D,DE43,2013-09-07,0.951986623536155
7D,DE43,2013-09-08,0.9503816710700315
7D,DE43,2013-09-09,0.9487767186039082
7D,DE43,2013-09-10,0.9471717661377848
7D,DE43,2013-09-11,0.9455668136716613
7D,DE43,2013-09-12,0.9439618612055378
7D,DE43,2013-09-13,0.9423569087394145
7D,DE43,2013-09-14,0.9407519562732909
7D,DE43,2013-09-15,0.9391470038071676
7D,DE43,2013-09-16,0.9375420513410441
7D,DE43,2013-09-17,0.9359370988749207
7D,DE43,2013-09-18,0.9343321464087972
7D,DE43,2013-09-19,0.9327271939426738
7D,DE43,2013-09-20,0.9311222414765504
7D,DE43,2013-09-21,0.9295172890104269
7D,DE43,2013-09-22,0.9279123365443035
7D,DE43,2013-09-23,0.9263073840781801
7D,DE43,2013-09-24,0.9247024316120566
7D,DE43,2013-09-25,0.9230974791459331
7D,DE43,2013-09-26,0.9214925266798099
7D,DE43,2013-09-27,0.9198875742136863
7D,DE43,2013-09-28,0.9182826217475628
7D,DE43,2013-09-29,0.9166776692814395
7D,DE43,2013-09-30,0.915072716815316
7D,DE43,2013-10-01,0.9134677643491925
7D,DE43,2013-10-02,0.9118628118830692
7D,DE43,2013-10-03,0.9102578594169456
7D,DE43,2013-10-04,0.9086529069508222
7D,DE43,2013-10-05,0.9070479544846989
7D,DE43,2013-10-06,0.9054430020185753
7D,DE43,2013-10-07,0.9038380495524518
7D,DE43,2013-10-08,0.9022330970863285
7D,DE43,2013-10-09,0.900628144620205
7D,DE43,2013-10-10,0.8990231921540816
"""

my_tag = AttribDict()
my_tag.namespace = ns
my_tag.value = csv_data


selected_inv.networks[0].stations[0].extra = AttribDict()
selected_inv.networks[0].stations[0].extra.gpsclockcorrection =my_tag

selected_inv.write('modified_inventory_select.xml', format='STATIONXML',
           nsmap={'GeoscienceAustralia': 'https://github.com/GeoscienceAustralia/hiperseis/xmlns/1.0'})

my_inv.write('modified_inventory.xml', format='STATIONXML')

Inventory created at 2017-09-05T16:57:36.000000Z
	Created by: ObsPy 1.0.2
		    https://www.obspy.org
	Sending institution: Geoscience Australia
	Contains:
		Networks (1):
			7D
		Stations (1):
			7D.DE43 (DE43)
		Channels (3):
			7D.DE43..BHZ, 7D.DE43..BHN, 7D.DE43..BHE


In [8]:
# print (my_inv.networks[0].stations)

In [9]:
print(selected_inv)

Inventory created at 2017-09-05T16:57:36.000000Z
	Created by: ObsPy 1.0.2
		    https://www.obspy.org
	Sending institution: Geoscience Australia
	Contains:
		Networks (1):
			7D
		Stations (1):
			7D.DE43 (DE43)
		Channels (3):
			7D.DE43..BHZ, 7D.DE43..BHN, 7D.DE43..BHE


# Reading the new modified stationxml file and make use of the gps correcton CSV data

In [10]:
our_new_station_xml= 'modified_inventory_select.xml'


our_inv = read_inventory(our_new_station_xml)
# print(our_inv.networks[0].stations[0].extra)

csv_str = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

In [11]:
print (csv_str)

net,sta,date,clock_correction
7D,DE43,2012-11-27,1.0398489013215846
7D,DE43,2012-11-28,0.9408504322549281
7D,DE43,2012-11-29,0.8418519631882714
7D,DE43,2012-11-30,0.7428534941216148
7D,DE43,2012-12-01,0.6438550250549583
7D,DE43,2012-12-02,0.5448565559883017
7D,DE43,2012-12-03,0.445858086921645
7D,DE43,2012-12-04,0.3468596178549885
7D,DE43,2012-12-05,0.247861148788332
7D,DE43,2012-12-06,0.14886267972167544
7D,DE43,2012-12-07,0.04986421065501878
7D,DE43,2012-12-08,-0.049134258411637766
7D,DE43,2012-12-09,-0.14813272747829442
7D,DE43,2012-12-10,-0.24713119654495086
7D,DE43,2012-12-11,-0.3461296656116075
7D,DE43,2012-12-12,-0.4451281346782642
7D,DE43,2012-12-13,-0.5441266037449206
7D,DE43,2012-12-14,-0.6431250728115772
7D,DE43,2012-12-15,-0.7421235418782337
7D,DE43,2012-12-16,-0.8411220109448904
7D,DE43,2012-12-17,-0.9401204800115468
7D,DE43,2012-12-18,-1.0391189490782036
7D,DE43,2012-12-19,-1.13811741814486
7D,DE43,2012-12-20,-1.2371158872115164
7D,DE43,2012-12-21,-1.3361143562781734
7D,D

In [12]:

import pandas as pd

In [13]:

df = pd.DataFrame([x.split(',') for x in csv_str.split('\n')][2:])
print(df.head())

    0     1           2                   3
0  7D  DE43  2012-11-28  0.9408504322549281
1  7D  DE43  2012-11-29  0.8418519631882714
2  7D  DE43  2012-11-30  0.7428534941216148
3  7D  DE43  2012-12-01  0.6438550250549583
4  7D  DE43  2012-12-02  0.5448565559883017


In [19]:
# This should be replaced by the next code using tempfile 

# CSV_FILE_NAME = 'temp_file.csv'  # Consider creating temp file, look example below
# with open(CSV_FILE_NAME, 'w') as outfile:
#     outfile.write(csv_str)
# df = pd.read_csv(CSV_FILE_NAME, sep=',', header=0)

In [20]:
import os, tempfile
tmp = tempfile.NamedTemporaryFile(delete=False)

try:
    print(tmp.name)
    with open(tmp.name, 'w') as outfile:
        outfile.write(csv_str)
    
    df = pd.read_csv(tmp.name, sep=',', header=0)
    
finally:
    #os.unlink(tmp.name) # remove the tmp file after use. 
    tmp.close()

/local/p25/fxz547/tmp/tmp7lqnd6mg


In [21]:
print (df.head())

  net   sta        date  clock_correction
0  7D  DE43  2012-11-27          1.039849
1  7D  DE43  2012-11-28          0.940850
2  7D  DE43  2012-11-29          0.841852
3  7D  DE43  2012-11-30          0.742853
4  7D  DE43  2012-12-01          0.643855


In [22]:
print (df.tail())

    net   sta        date  clock_correction
240  7D  DE43  2013-10-06          0.905443
241  7D  DE43  2013-10-07          0.903838
242  7D  DE43  2013-10-08          0.902233
243  7D  DE43  2013-10-09          0.900628
244  7D  DE43  2013-10-10          0.899023
